# Классификация комментариев

Интернет магазину необходимо найти токсичные комментарии и отправлять их на модерацию. Неоходимо предоставить модель классификации комментраиев на позитивные и негативные с метрикой качества f1 не меньше 0,75

Порядок работы представлен ниже:

<h1>Порядок работы<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Лемматизация-и-подсчеты-длины-коментария" data-toc-modified-id="Лемматизация-и-подсчеты-длины-коментария-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизация и подсчеты длины коментария</a></span></li><li><span><a href="#Проверка-гиппотезы" data-toc-modified-id="Проверка-гиппотезы-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Проверка гиппотезы</a></span></li><li><span><a href="#Подготовка-целевого-признака-и-фичей" data-toc-modified-id="Подготовка-целевого-признака-и-фичей-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Подготовка целевого признака и фичей</a></span></li><li><span><a href="#Итог" data-toc-modified-id="Итог-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Итог</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#подготовка-pipeline" data-toc-modified-id="подготовка-pipeline-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>подготовка pipeline</a></span></li><li><span><a href="#GridsearchCV" data-toc-modified-id="GridsearchCV-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>GridsearchCV</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Итоги-обучения" data-toc-modified-id="Итоги-обучения-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Итоги обучения</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

In [1]:
conda install -c conda-forge imbalanced-learn


Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::alembic==1.4.3=pyh9f0ad1d_0
  - conda-forge/noarch::jsonschema==3.2.0=py_2
  - conda-forge/linux-64::pycurl==7.43.0.6=py38hd8bd627_0
  - conda-forge/noarch::jupyter_telemetry==0.0.5=py_0
  - conda-forge/noarch::jupyter_client==6.1.7=py_0
  - conda-forge/linux-64::jupyterhub-base==1.2.1=py38h578d9bd_0
  - conda-forge/linux-64::notebook==6.1.4=py38h32f6830_1
  - conda-forge/linux-64::pyrsistent==0.17.3=py38h1e0a361_1
  - conda-forge/linux-64::terminado==0.9.1=py38h32f6830_1
  - conda-forge/linux-64::argon2-cffi==20.1.0=py38h1e0a361_2
  - conda-forge/noarch::jupyterlab_server==1.2.0=py_0
  - conda-forge/linux-64::pyzmq==19.0.2=py38ha71036d_2
  - conda-forge/linux-64::ruamel.yaml==0.16.12=py38h1e0a361_1
  - conda-forge/noarch::nbclient==0.5.1=py_0
  - conda-forge/linux-64::jedi==0.17.2=py38h32f6830_1
  - conda-forge/

In [2]:
pip install -U imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 206 kB 868 kB/s eta 0:00:01
     |████████████████████████████████| 22.3 MB 96 kB/s  eta 0:00:01    |██████████▎                     | 7.2 MB 4.0 MB/s eta 0:00:04
Note: you may need to restart the kernel to use updated packages.


Используемые библиотеки

In [3]:
#обработка данных
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer 
import re
import nltk
from nltk.corpus import stopwords 
from scipy import stats as st
from sklearn.feature_extraction.text import TfidfVectorizer
import copy
from scipy import stats as st
from sklearn.compose import ColumnTransformer


#поиск решений

from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from imblearn.pipeline import Pipeline
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#модели

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

#метрики
from sklearn.metrics import f1_score


#таймер
import timeit



In [4]:
import warnings

warnings.filterwarnings('ignore')

## Подготовка

### Загрузка данных

In [5]:
data=pd.read_csv('')
data.head(2)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0


Рассмотрим информацию о данных

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет, рассмтрим на дубликаты.

In [7]:
data.duplicated().sum()

0

Дубликатов тоже нет.


По типу задача представляет классификацию, с предсказанием двух классов. Рассмотрим как распределены классы в представленном датасете

In [8]:
data.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Очевидный дисбаланс классов. Поэтому необходимо будет их сбалансировать.Балансировку будем проводить как в сторону максимального значения (upsampling) так и в сторону миниимального значения (downsampling). данную преобработку будем проводить на этапе обучения

Замечено, что при обрабоке данных с 110000 по 120000 почему то улетает ядро, поэтому я решил их просто убрать

In [9]:
data_for_preprocessing=data.drop(index=[i for i in range(110000,120000,1)]).reset_index(drop=True)

### Лемматизация и подсчеты длины коментария

Проведем лемматизацию и токенизацию для этого воспользуемся функцией func_lem()

In [10]:
corpus = data_for_preprocessing['text']

In [11]:
lemmatizer = WordNetLemmatizer()

In [12]:
def func_lem(x):
    withhout_prep=x
    pattern=re.sub(r"[^A-Za-z0-9!@#\$%\^\&\*_\-,\.' ]",' ' ,withhout_prep)
    split=pattern.split()
    lemm=[lemmatizer.lemmatize(w) for w in split]
    out=" ".join(lemm)
    return out

In [13]:
lemm_text=pd.Series(corpus).apply(lambda x: func_lem(x))

Посчитаем сколько слов в каждом коментарии

In [14]:
def count_word(x):
    withhout_prep=x
    pattern=re.sub(r"[^A-Za-z0-9!@#\$%\^\&\*_\-,\.' ]",' ' ,withhout_prep)
    split=pattern.split()
    count=len(split)
    return count

In [15]:
count_word=pd.Series(corpus).apply(lambda x: count_word(x))

И добавим это как новый признак

In [16]:

data_for_learn=pd.DataFrame({'toxic':data_for_preprocessing['toxic'],'text_lem':lemm_text,'count_word':count_word})
data_for_learn.head(2)

,toxic,text_lem,count_word
0,0,Explanation Why the edits made under my userna...,43
1,0,D'aww! He match this background colour I'm see...,18


### Проверка гиппотезы

Рассмотрим сколько слов  в токсичном коментарии

In [17]:
data_for_learn.query('toxic==1')['count_word'].describe()

count    15252.000000
mean        53.365854
std        108.423535
min          2.000000
25%         11.000000
50%         23.000000
75%         51.000000
max       1403.000000
Name: count_word, dtype: float64

В токсичных коментариях преобладают короткие сообщения. Поэтому выдвеним такую гиппотезу:  средняя длина токсичного коментария меньше 70. Где нулевая теория будет: Средняя длина = 70, а альтернативная меньше 70

In [18]:
tox = data_for_learn.query('toxic==1')['count_word']

mean_word_in_tox = 70

alpha = .05 

results = st.ttest_1samp(
    tox, 
    mean_word_in_tox)

print('p-значение: ', results.pvalue / 2)

# тест односторонний влево:
# отвергаем гипотезу только если выборочное среднее значимо меньше предполагаемого значения
if (results.pvalue / 2 < alpha) and (tox.mean() < mean_word_in_tox):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

p-значение:  1.8929265575784114e-79
Отвергаем нулевую гипотезу


Следовательно отвегаем нулевую теорию, и предполгаем, что длина токсичного коментария меньше 70 слов.

In [19]:
data_for_learn=data_for_learn.query('count_word<70')
data_for_learn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108760 entries, 0 to 149570
Data columns (total 3 columns):
toxic         108760 non-null int64
text_lem      108760 non-null object
count_word    108760 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.3+ MB


In [20]:
data_for_learn.head(2)

,toxic,text_lem,count_word
0,0,Explanation Why the edits made under my userna...,43
1,0,D'aww! He match this background colour I'm see...,18


Данные готовы к подготвке целевого признака и фичей

### Подготовка целевого признака и фичей

Разделим данные на фичи и целевой признак

In [21]:

features=data_for_learn.drop('toxic', axis=1)
target=data_for_learn['toxic']

Разделим данные на тренировочную и тестовую выборку в соотношении 80/20

In [22]:
features_train,features_test,target_train,target_test=(train_test_split(
features, target, test_size=0.2, random_state=42,stratify=target))



Данные разбиты 

### Итог

Данные подготовлены теперь можно переходить к обучению

## Обучение

### подготовка pipeline

создавать признаки будем с помощью tfidf преобразования, поэтому необходимо создать стоп слова

In [23]:
#nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

Создадим pipeline в которых шаги можно описать так: трансформацию с помощью TfidfVectorizer, а затем уже одна из трех моделей(логистическая регрессия, случайные деревья и lgbmclassificator). Также будем выбирать исходя из upsampling и downsampling

In [24]:
column_trans = ColumnTransformer([
    ('text_trans',TfidfVectorizer(stop_words=stop_words,min_df=0.0001,ngram_range=(1,1)),'text_lem')
])

Pipeline загружен из библиотеки imbalanced, реалтзованной для дисбаланса классов. 

In [25]:
#линейная регрессия
pipe_lr_upsampling=(
    Pipeline([('tfidf',
 column_trans),
              ('upsampling',
SMOTE(random_state=42)),
               ('est',
LogisticRegression(random_state=42,class_weight='balanced',solver='liblinear'))])
              )

In [26]:
#линейная регрессия
pipe_lr_downsampling=(
    Pipeline([('tfidf',
column_trans),
              ('downsampling',
RandomUnderSampler(sampling_strategy=0.5)),
               ('est',
LogisticRegression(random_state=42,class_weight='balanced',solver='liblinear'))])
              )

In [27]:
#случайный деревья
pipe_rf_upsampling=(
    Pipeline([('tfidf',
 column_trans),
              ('upsampling',
SMOTE(random_state=42)),
               ('estrf',
RandomForestClassifier(random_state=42))])
              )

In [28]:
#случайный деревья
pipe_rf_downsampling=(
    Pipeline([('tfidf',
 column_trans),
              ('downsampling',
RandomUnderSampler(sampling_strategy=0.5)),
               ('estrf',
RandomForestClassifier(random_state=42))])
              )

In [29]:
# lgbm
pipe_lgbm_upsampling=(
    Pipeline([('tfidf',
 column_trans),
              ('upsampling',
SMOTE(random_state=42)),
               ('lgbm',
lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', class_weight = 'balanced',learning_rate=0.05))])
              )

In [30]:
# lgbm
pipe_lgbm_downsampling=(
    Pipeline([('tfidf',
 column_trans),
              ('downsampling',
RandomUnderSampler(sampling_strategy=0.5)),
               ('lgbm',
lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', class_weight = 'balanced',learning_rate=0.05))])
              )

Наилучшую модель будем искать с помощью кроссвалидации

### GridsearchCV

Создадим функцию которая возвращает наилучший pipeline

In [31]:
def func_gs(pip,param,features,target):
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    gs=GridSearchCV(estimator=pip,param_grid=param,cv=kf,scoring='f1')
    gs.fit(features,target)
    print(f1_score(target,gs.best_estimator_.predict(features)))
    print(f1_score(target_test,gs.best_estimator_.predict(features_test)))
    return gs.best_estimator_

обозначим парметры которые будут подбираться для каждой из моделей

In [32]:
#логистическая регрессия: здесь я проводил на таких гипперпарамеьрах, однако для того, 
#что бы быстрее прогружать тетрадку я их закоментил и оставли только наилучший вариант
#param_lr={'est__max_iter':[30,50,75,100],'tfidf__ngram_range':[(1,1),(2,2),(1,3)]}
#param_lr={'est__max_iter':[100],'tfidf__ngram_range':[(1,1)]}
param_lr={'est__max_iter':[30,50,75,100]}

In [33]:
param_rf={'estrf__n_estimators':[75,100],
          'estrf__max_depth':[30,50],
          'tfidf__ngram_range':[(1,1),(2,2)],
          
         }

In [34]:
# Здесь я поступил точно также
# param_lgbm={'lgbm__n_estimators':[100,500],
#            'lgbm__max_depth':[10,30,50]}
param_lgbm={'lgbm__n_estimators':[500],
            'lgbm__max_depth':[10,30,50]}

### Обучение

In [35]:
#данная подборка давал плохой рещультат, поэтому я его закоментил и дальше не стал пускать
lr_up=func_gs(pipe_lr_upsampling,param_lr,features_train,target_train)

0.7490583410061102
0.6893865628042843


In [36]:
lr_dw=func_gs(pipe_lr_downsampling,param_lr,features_train,target_train)

0.7776318576572758
0.7343617562298694


In [37]:
#случайные деревья таже себя повели плохо, поэтому для ускорения процесса я их не запускаю
#rf_up=func_gs(pipe_rf_upsampling,param_rf,features_train,target_train)

In [38]:
#rf_dw=func_gs(pipe_rf_downsampling,param_rf,features_train,target_train)


In [39]:
#upsampling для lgbm оказался неудачным и очень длительным, пожтому я также его закоментил
#lgbm_up=func_gs(pipe_lgbm_upsampling,param_lgbm,features_train,target_train)

In [40]:
lgbm_dw=func_gs(pipe_lgbm_downsampling,param_lgbm,features_train,target_train)

0.8003967504250898
0.7610287216069083


### Итоги обучения

Создаим сводную таблицу куда будем вносить все измеренные значения

In [41]:
table=pd.DataFrame(columns=['модель','f1 тренировка','f1 тест','время предсказания'])


In [42]:
def full_table(pipe,model):
    start_time=timeit.default_timer()
    pred=pipe.predict(features_test)
    time_predict=timeit.default_timer()-start_time
    f1_train=f1_score(target,pipe.predict(features))
    f1_test=f1_score(target_test,pred)
    return [model,f1_train,f1_test,time_predict]
    

In [43]:
a=pd.Series(full_table(lr_up,'логистическая up'))
table=table.append({'модель':a[0],
                 'f1 тренировка':a[1],
                 'f1 тест':a[2],
                    'время предсказания':a[3]
                   },
                  ignore_index=True)


In [44]:
a=pd.Series(full_table(lr_dw,'логистическая dw'))
table=table.append({'модель':a[0],
                 'f1 тренировка':a[1],
                 'f1 тест':a[2],
                    'время предсказания':a[3]
                   },
                  ignore_index=True)

In [45]:
# a=pd.Series(full_table(rf_up,'случайные деревья up'))
# table=table.append({'модель':a[0],
#                  'f1 тренировка':a[1],
#                  'f1 тест':a[2],
#                     'время предсказания':a[3]
#                    },
#                   ignore_index=True)


In [46]:
# a=pd.Series(full_table(rf_dw,'случайные деревья dw'))
# table=table.append({'модель':a[0],
#                  'f1 тренировка':a[1],
#                  'f1 тест':a[2],
#                     'время предсказания':a[3]
#                    },
#                   ignore_index=True)


In [47]:
# a=pd.Series(full_table(lgbm_up,'lgbm up'))
# table=table.append({'модель':a[0],
#                  'f1 тренировка':a[1],
#                  'f1 тест':a[2],
#                     'время предсказания':a[3]
#                    },
#                   ignore_index=True)


In [48]:
a=pd.Series(full_table(lgbm_dw,'lgbm dw'))
table=table.append({'модель':a[0],
                 'f1 тренировка':a[1],
                 'f1 тест':a[2],
                    'время предсказания':a[3]
                   },
                  ignore_index=True)


In [49]:
table

,модель,f1 тренировка,f1 тест,время предсказания
0,логистическая up,0.736825,0.689387,0.728063
1,логистическая dw,0.768980,0.734362,0.655876
2,lgbm dw,0.792483,0.761029,3.129592


## Выводы

Задача по поиску необходимой модели была выполнена  на модели логистической регрессия с метрикой качества f1=0.75

Предобработка данных:

На первом этапе было изъято около 1000 объектов, так как они каким то образом разрушали ядро и не давали произвести вычисления.
на следующем этапе был произведена лемматизация, в результате которой удалось вычесть количество слов в каждом  коментарии. Данная процедура позволила вынести теорию о том, что средняя длина негативного комената меньше 70 слов. Данную теорию подвердили и удалили все коментарии длиннее этого значения.

Также присутвовал дисбаланс классов. Решение этой проблемы было проведено путем upsampling и downsampling.

Обучение:

Для обучения использовались три модели: логистическая регрессия, случайные деревья и lgbm. в результате обучения наихудшее параметры показали случанын деревья. Наилучашая модель lgbm.

Стоит отметить, что одним из параметров во всех моделях была количесвто n-gramm. и у всех моделей наилучшее значения достигали при n-gramm=1 слово. Таким образом можно сказать, что модели реагируют на одни конкретные слова, не вникая в смысл.

